In [1]:
import pandas as pd
import yfinance as yf

In [2]:
def get_ticker(ticker):
    ticker = yf.Ticker(ticker)

    return ticker


ticker = get_ticker('AAPL')

In [3]:
def obtain_cashflow(ticker):

    cashflow = ticker.cashflow

    return cashflow


aapl_cashflow = obtain_cashflow(ticker)
aapl_cashflow

,2023-09-30,2022-09-30,2021-09-30,2020-09-30,2019-09-30
Free Cash Flow,99584000000.0,111443000000.0,92953000000.0,73365000000.0,NaN
Repurchase Of Capital Stock,-77550000000.0,-89402000000.0,-85971000000.0,-72358000000.0,NaN
Repayment Of Debt,-11151000000.0,-9543000000.0,-8750000000.0,-12629000000.0,NaN
Issuance Of Debt,5228000000.0,5465000000.0,20393000000.0,16091000000.0,NaN
Issuance Of Capital Stock,NaN,NaN,1105000000.0,880000000.0,781000000.0
Capital Expenditure,-10959000000.0,-10708000000.0,-11085000000.0,-7309000000.0,NaN
Interest Paid Supplemental Data,3803000000.0,2865000000.0,2687000000.0,3002000000.0,NaN
Income Tax Paid Supplemental Data,18679000000.0,19573000000.0,25385000000.0,9501000000.0,NaN
End Cash Position,30737000000.0,24977000000.0,35929000000.0,39789000000.0,NaN
Beginning Cash Position,24977000000.0,35929000000.0,39789000000.0,50224000000.0,NaN


# Break Down the Cash Flow Sections

a. Operating Activities
This section tells you how much cash the company generates from its core operations. Look at:

- Net Income: This gives you a baseline of the profitability.
- Depreciation & Amortization: Non-cash expenses that reduce net income but do not affect cash flow.
- Changes in Working Capital: Variations in accounts receivable, inventory, and payable

b. Investing Activities
This section reflects where Apple is investing its capital, often in:

- Capital Expenditures (CapEx): Investments in physical assets like factories, stores, or equipment. A rising CapEx might indicate expansion.
- Purchases/Sales of Investments: Apple holds a large portfolio of marketable securities. Analyze how much of their cash flow is tied up in investments versus being deployed in business operations.

c. Financing Activities
This shows how Apple finances its operations and returns cash to shareholders:

- Share Buybacks: Apple has been repurchasing a large number of shares.
- Dividend Payments: Regular cash outflow to shareholders.
- Debt Issuance/Repayment: Analyze whether Apple is issuing debt or repaying it. Despite large cash reserves, Apple has issued debt due to historically low-interest rates.

# Key Ratios and Metrics

- Free Cash Flow (FCF):  
FCF = Operating Cash Flow - Capital Expenditures  
This is a key metric to see how much cash is available after Apple covers its basic operational expenses.
- Cash Conversion Ratio (CCR):  
CCR = Operating Cash Flow / Net Income  
This ratio shows how well Apple converts its net income into cash.